In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input, Attention, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
import numpy as np

In [2]:
with open('data.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        word_list = [line.strip().lower() for line in lines]

# Sample data for outputs
vocab = word_list
characters = set(''.join(word_list))

# Encoding dictionaries
char_to_id = {char: i + 1 for i, char in enumerate(sorted(characters))}
char_to_id[" "] = 0
word_to_id = {word: i for i, word in enumerate(vocab)}

# Inverse for decoding
id_to_word = {i: word for word, i in word_to_id.items()}

max_char_length = max(len(word) for word in word_list)  # Maximum length of words in `data`

# Convert input words to encoded characters
X=[]
Y=[]
for word in word_list:
    for i in range(1,len(word)):
        x=[]
        for char in word[:i]:
            x.append(char_to_id[char])
        X.append(x)
        Y.append(word_to_id[word])

X_padded = pad_sequences(X, maxlen=max_char_length, padding='post')
X_padded = np.array(X_padded)
Y = np.array(Y)

In [3]:
# Hyperparameters
vocab_size = len(vocab)  # Number of words in the vocabulary
char_vocab_size = 256  # Assuming ASCII characters for simplicity
embedding_dim = 64
rnn_units = 128

# Model Definition
input_chars = Input(shape=(None,), dtype='int32')
char_embeddings = Embedding(input_dim=char_vocab_size, output_dim=embedding_dim)(input_chars)
rnn_out, state = SimpleRNN(units=rnn_units, return_sequences=True, return_state=True)(char_embeddings)

# Attention mechanism
query = tf.expand_dims(state, 1)  # Use the last RNN state as the query
attention_output, attention_weights = Attention(use_scale=True)([query, rnn_out], return_attention_scores=True)
flattened_output = Flatten()(attention_output)

# Output layer to predict the word index
word_pred = Dense(vocab_size, activation='softmax')(flattened_output)

model = Model(inputs=input_chars, outputs=word_pred)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 64)             16384     ['input_1[0][0]']             
                                                                                                  
 simple_rnn (SimpleRNN)      [(None, None, 128),          24704     ['embedding[0][0]']           
                              (None, 128)]                                                        
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 128)               0         ['simple_rnn[0][1]']    

In [4]:
model.fit(X_padded, Y, epochs=1000, batch_size=64)


Epoch 1/1000


874/874 [==============================] - 9s 9ms/step - loss: 8.6043 - accuracy: 0.0010
Epoch 2/1000
874/874 [==============================] - 8s 10ms/step - loss: 7.4011 - accuracy: 0.0025
Epoch 3/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.8887 - accuracy: 0.0040
Epoch 4/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.5831 - accuracy: 0.0061
Epoch 5/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.3709 - accuracy: 0.0066
Epoch 6/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.2199 - accuracy: 0.0080
Epoch 7/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.0481 - accuracy: 0.0106
Epoch 8/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.9605 - accuracy: 0.0107
Epoch 9/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.4283 - accuracy: 0.0077
Epoch 10/1000
874/874 [==============================] - 8s 10ms/step - 

874/874 [==============================] - 8s 10ms/step - loss: 4.5046 - accuracy: 0.1140
Epoch 154/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.3995 - accuracy: 0.1171
Epoch 155/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.3242 - accuracy: 0.1225
Epoch 156/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.2393 - accuracy: 0.1279
Epoch 157/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.2812 - accuracy: 0.1273
Epoch 158/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.2434 - accuracy: 0.1293
Epoch 159/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.3827 - accuracy: 0.1199
Epoch 160/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.4140 - accuracy: 0.1159
Epoch 161/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.4235 - accuracy: 0.1178
Epoch 162/1000
874/874 [==============================] - 9s 10ms/step

874/874 [==============================] - 9s 10ms/step - loss: 6.0048 - accuracy: 0.0393
Epoch 310/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.0500 - accuracy: 0.0387
Epoch 311/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.0751 - accuracy: 0.0381
Epoch 312/1000
874/874 [==============================] - 9s 10ms/step - loss: 6.0281 - accuracy: 0.0384
Epoch 313/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.0158 - accuracy: 0.0393
Epoch 314/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.0021 - accuracy: 0.0413
Epoch 315/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.9955 - accuracy: 0.0412
Epoch 316/1000
874/874 [==============================] - 9s 10ms/step - loss: 6.2303 - accuracy: 0.0369
Epoch 317/1000
874/874 [==============================] - 8s 10ms/step - loss: 6.3704 - accuracy: 0.0383
Epoch 318/1000
874/874 [==============================] - 8s 10ms/step

874/874 [==============================] - 8s 10ms/step - loss: 5.5633 - accuracy: 0.0854
Epoch 466/1000
874/874 [==============================] - 9s 10ms/step - loss: 5.4754 - accuracy: 0.0913
Epoch 467/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.4524 - accuracy: 0.0945
Epoch 468/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.4181 - accuracy: 0.0952
Epoch 469/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.3976 - accuracy: 0.0960
Epoch 470/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.3925 - accuracy: 0.0964
Epoch 471/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.3948 - accuracy: 0.0973
Epoch 472/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.3829 - accuracy: 0.1011
Epoch 473/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.3781 - accuracy: 0.0996
Epoch 474/1000
874/874 [==============================] - 8s 10ms/step

874/874 [==============================] - 9s 10ms/step - loss: 5.2531 - accuracy: 0.1258
Epoch 622/1000
874/874 [==============================] - 9s 10ms/step - loss: 5.1864 - accuracy: 0.1287
Epoch 623/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.2170 - accuracy: 0.1292
Epoch 624/1000
874/874 [==============================] - 9s 10ms/step - loss: 5.2476 - accuracy: 0.1261
Epoch 625/1000
874/874 [==============================] - 9s 11ms/step - loss: 5.1502 - accuracy: 0.1339
Epoch 626/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.1653 - accuracy: 0.1321
Epoch 627/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.2109 - accuracy: 0.1271
Epoch 628/1000
874/874 [==============================] - 8s 10ms/step - loss: 5.1617 - accuracy: 0.1324
Epoch 629/1000
874/874 [==============================] - 9s 10ms/step - loss: 5.2916 - accuracy: 0.1249
Epoch 630/1000
874/874 [==============================] - 8s 10ms/step

874/874 [==============================] - 8s 10ms/step - loss: 4.7813 - accuracy: 0.1448
Epoch 778/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.7589 - accuracy: 0.1468
Epoch 779/1000
874/874 [==============================] - 9s 10ms/step - loss: 4.6842 - accuracy: 0.1525
Epoch 780/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.6565 - accuracy: 0.1507
Epoch 781/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.6825 - accuracy: 0.1531
Epoch 782/1000
874/874 [==============================] - 8s 10ms/step - loss: 4.8347 - accuracy: 0.1442
Epoch 783/1000
874/874 [==============================] - 9s 10ms/step - loss: 4.8187 - accuracy: 0.1438
Epoch 784/1000
874/874 [==============================] - 9s 10ms/step - loss: 4.8588 - accuracy: 0.1412
Epoch 785/1000
874/874 [==============================] - 9s 10ms/step - loss: 4.7850 - accuracy: 0.1413
Epoch 786/1000
874/874 [==============================] - 9s 10ms/step

874/874 [==============================] - 8s 10ms/step - loss: 2.9364 - accuracy: 0.2653
Epoch 934/1000
874/874 [==============================] - 9s 10ms/step - loss: 2.9361 - accuracy: 0.2654
Epoch 935/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9332 - accuracy: 0.2668
Epoch 936/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9323 - accuracy: 0.2645
Epoch 937/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9321 - accuracy: 0.2674
Epoch 938/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9328 - accuracy: 0.2654
Epoch 939/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9338 - accuracy: 0.2638
Epoch 940/1000
874/874 [==============================] - 8s 10ms/step - loss: 2.9332 - accuracy: 0.2627
Epoch 941/1000
874/874 [==============================] - 9s 10ms/step - loss: 2.9316 - accuracy: 0.2639
Epoch 942/1000
874/874 [==============================] - 9s 10ms/step

In [5]:
model.save('03102024Final3_2.h5')
#model = load_model('03102024Final3.h5')

C:\Users\Stude\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
input_chars = "ac"  # Example input
input_ids = [char_to_id.get(char, 0) for char in input_chars]
input_padded = pad_sequences([input_ids], maxlen=max_char_length, padding='post')

# Make a prediction
prediction = model.predict(input_padded)
# Get the top 3 word IDs and their probabilities
top_3_indices = np.argsort(prediction[0])[-3:][::-1]  # Sort and get top 3 indices
top_3_probs = np.sort(prediction[0])[-3:][::-1]  # Sort and get top 3 probabilities
top_3_words = [id_to_word[idx] for idx in top_3_indices]
top_3_percentages = [f"{prob*100:.2f}%" for prob in top_3_probs]
for word, percentage in zip(top_3_words, top_3_percentages):
    print(f"Word: {word}, Softmax: {percentage}")

1/1 [==============================] - 0s 16ms/step
Word: academic, Softmax: 3.64%
Word: achieve, Softmax: 2.42%
Word: acquired, Softmax: 2.40%
